In [1]:
import investpy as ip
import pandas as pd
from tqdm import tqdm
from finquant.portfolio import build_portfolio
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'finquant'

In [ ]:
def as_number(s):
    if isinstance(s,float):
        return(s)
    else:
        return(float(s.replace(' ','')))

In [ ]:
def theoretical_price_1(x):
    EPS = as_number(x.EPS[0])
    PE = as_number(x['P/E Ratio'][0])
    theoretical_price = PE * EPS
    return(theoretical_price)

def theoretical_price_2(x):
    mcap = as_number(x['Market Cap'][0])
    shares_outstanding = as_number(x['Shares Outstanding'][0])
    theoretical_price = mcap / shares_outstanding
    return(theoretical_price)

def theoretical_price_3(x):
    temp = x['Dividend (Yield)'][0].split(' (')
    div = as_number(temp[0])
    div_yield = as_number(temp[1][:4]) / 100 #because of % symbol in original data
    theoretical_price = div/div_yield
    return(theoretical_price)

In [16]:
tickers = list(ip.get_stocks(country='brazil').symbol)
tickers
#tickers = tickers[0:5]

['ABCB4',
 'AGRO3',
 'RAIL3',
 'ALPA3',
 'ALPA4',
 'ALSO3',
 'AMAR3',
 'ABEV3',
 'ADHM3',
 'ARZZ3',
 'BBAS3',
 'BBDC3',
 'BBDC4',
 'BBRK3',
 'BEEF3',
 'BPHA3',
 'BPAN4',
 'BRAP3',
 'BRAP4',
 'BRFS3',
 'APER3',
 'BRKM3',
 'BRKM5',
 'BRML3',
 'BRPR3',
 'BRSR6',
 'OIBR3',
 'OIBR4',
 'BTOW3',
 'B3SA3',
 'CAMB4',
 'CARD3',
 'CCPR3',
 'CCRO3',
 'CEDO4',
 'CEED3',
 'CESP6',
 'CGAS5',
 'CGRA4',
 'CIEL3',
 'CMIG3',
 'CMIG4',
 'COCE5',
 'CPFE3',
 'CPLE3',
 'CPLE6',
 'CRDE3',
 'CSAN3',
 'CSMG3',
 'CSNA3',
 'LIQO3',
 'CTNM4',
 'CYRE3',
 'DASA3',
 'DIRR3',
 'DTEX3',
 'ECOR3',
 'EEEL3',
 'ELEK4',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENBR3',
 'ENGI4',
 'EQTL3',
 'YDUQ3',
 'ETER3',
 'EUCA4',
 'EVEN3',
 'EZTC3',
 'FESA4',
 'FHER3',
 'TASA4',
 'FLRY3',
 'FRIO3',
 'TIET3',
 'TIET4',
 'GFSA3',
 'GGBR3',
 'GGBR4',
 'GOAU3',
 'GOAU4',
 'GOLL4',
 'GRND3',
 'GSHP3',
 'HBOR3',
 'HGTX3',
 'PRIO3',
 'HYPE3',
 'IDNT3',
 'IDVL4',
 'IGTA3',
 'MEAL3',
 'INEP3',
 'INEP4',
 'ITSA3',
 'ITSA4',
 'ITUB3',
 'ITUB4',
 'JBSS3',


In [17]:
info = ip.get_stock_information(tickers[0],'brazil')

In [18]:
df = pd.DataFrame([], columns = ['p','tp1','tp2','tp3'], index=tickers)
success, fail = 0, 0
for t in tqdm(tickers):
    try:
        info = ip.get_stock_information(t,'brazil')
        df.loc[t,'p'] = as_number(info['Open'][0])
        df.loc[t, 'tp1'] = theoretical_price_1(info)
        df.loc[t, 'tp2'] = theoretical_price_2(info)
        df.loc[t, 'tp3'] = theoretical_price_3(info)
        success += 1
    except:
        fail += 1

print(f'{success}, {fail}')

100%|████████████████████████████████████████████████████████████████████████████████| 749/749 [38:56<00:00,  3.12s/it]

0, 749


In [19]:
df['tp_agg'] = df.loc[:,['tp1','tp2','tp3']].apply(lambda x: min(x), axis=1)
df['expected_increase (%)'] = (df.tp_agg - df.p)/df.p * 100
df = df.dropna(axis=0)

In [20]:
df = df.sort_values('expected_increase (%)', ascending=False)
df.head(10)

,p,tp1,tp2,tp3,tp_agg,expected_increase (%)


In [21]:
stocks = df.head(5).index

In [22]:
prices = pd.DataFrame([], columns = stocks)
for s in tqdm(stocks):
    prices[s] = ip.get_stock_recent_data(s, 'brazil').Close

0it [00:00, ?it/s]


In [23]:
prices.head()

""


In [24]:
pf = build_portfolio(data = prices)
pf = pf.mc_optimisation(num_trials=5000)[0].loc['Max Sharpe Ratio']

alloc = pd.DataFrame(pf).reset_index()
alloc.columns = ['Name','Allocation']

pf = build_portfolio(data=prices,pf_allocation=alloc)

NameError: name 'build_portfolio' is not defined

In [ ]:
pf.properties()